LOAD DATA 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
data_path ='data/Task 2/sample_1a2bc3012c9defca.npz'


In [2]:
info = np.load(data_path,allow_pickle=True)
all_agent_trajs        = info['all_agent']    
all_agent_trajs.shape  #[24 agents, 91 timesteps the interval between each timestep is 0.1 second,\
    #10 denotes [center_x, center_y, center_z, boundingbox_x, boundingbox_y, boundingbox_z, heading, vel_x, vel_y, valid] ]
#the valid flag is either 0 or 1. 1 means the valid, 0 means invalid
obj_types              = info['object_type']
lane_polylines         = info['lane']           #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
road_polylines         = info['road_polylines'] #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
crosswalk_polylines    = info['crosswalk']      #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
speed_bump_polylines   = info['speed_bump']     #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
stop_signs_polylines   = info['stop_sign']      #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
drive_way_polylines    = info['drive_way']      #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]


We select the 11th timestep as the current timestep 

In [3]:
all_agent_current = all_agent_trajs[:,10]
all_agent_current.shape

(24, 10)

In [4]:
#ground truth for the future 1s,2s,3s can be easily derived by :
predict_horizon = 30 #20,30
all_gt_future=all_agent_trajs[:,11:11+predict_horizon]
all_gt_future.shape

(24, 30, 10)

Index of target agnet to preict

In [5]:
tracks        = info['predict_list']  
tracks
agents_to_predict = all_agent_trajs[tracks]
agents_to_predict.shape

(8, 91, 10)

In [6]:
"""
    1. Check if the data is valid in the first 11 timesteps (1 second)
    2. Global variables and class initialization
    3. ADE and FDE calculation function definition
"""

# 1. Check if the data is valid in the first 11 timesteps (1 second)
data_to_check = agents_to_predict[:, :11, :]  # shape [agents, 10, features]
for agent_index in range(data_to_check.shape[0]):
    for time_index in range(11):
        # If the data is invalid, replace it with nan
        if data_to_check[agent_index, time_index, 9] == 0: 
            data_to_check[agent_index, time_index, :9] = np.nan
    # Calculate the mean of the valid data
    mean = np.nanmean(data_to_check[agent_index, :, :9], axis=0)
    for time_index in range(11):
        # If the data is invalid, replace it with the mean
        if data_to_check[agent_index, time_index, 9] == 0:
            data_to_check[agent_index, time_index, :9] = mean
# Replace the first 10 timesteps with the corrected data
agents_to_predict[:, :11, :9] = data_to_check[:, :, :9]

# 2. Global variables and class initialization
@dataclass
class PredictionModel:
    name: str
    prediction: np.ndarray
    ades: np.ndarray
    fdes: np.ndarray
    distances: np.ndarray

his_start = 0
timestep = 0.1
predict_horizon = 30
current_positions = agents_to_predict[:,10]
gt_future_position = agents_to_predict[:,11:11+predict_horizon,:2]
predicted_trajectories = np.zeros((len(agents_to_predict),predict_horizon,2)) # shape (8, 30, 2)
# Average velocity in first 11 timesteps
vel_x_avg = np.mean(agents_to_predict[:,his_start:11,7], axis=1) # shape (8, 30)
vel_y_avg = np.mean(agents_to_predict[:,his_start:11,8], axis=1) # shape (8, 30)
# Average acceleration in first 11 timesteps
acc_x = np.diff(agents_to_predict[:,his_start:11,7], axis=1)/timestep # shape (8, 9)
acc_y = np.diff(agents_to_predict[:,his_start:11,8], axis=1)/timestep # shape (8, 9)
acc_x_avg = np.mean(acc_x, axis=1)
acc_y_avg = np.mean(acc_y, axis=1)

# Calculate the predicted trajectories for each agent
def ADE_FDE_calculation(predicted_trajectories):
    distances = np.linalg.norm(predicted_trajectories-gt_future_position,axis=2) # shape (8, 30)
    ades = np.mean(distances, axis=1)
    fdes = distances[:,-1]

    mean_ade = np.round(np.mean(ades), 4)
    mean_fde = np.round(np.mean(fdes), 4)

    final_ades = np.round(ades, 4)
    final_fdes = np.round(fdes, 4)
    
    for i, track in enumerate(tracks):
        print(f"Agent {track} - ADE: {final_ades[i]} FDE: {final_fdes[i]}")
    print(f'\nMean ADE: {mean_ade} Mean FDE: {mean_fde}')

    return final_ades, final_fdes, distances


The homework should be done on these 8 agents.

Q1.Using constant velocity model to predict the future (1s,2s,3s) trajectories of the target agent and calculate the ADE and FDE.  

In [7]:
"""
    #Constant Velocity Model#

    cv_predicted_trajectories instance is created 
    by using the average velocity of the first 
    10 timesteps to predict the future positions.
"""

# Use the average velocity to predict the future positions
for t in range(predict_horizon):
    predicted_trajectories[:, t, 0] = current_positions[:, 0] + vel_x_avg * (t+1) * timestep  # shape [8, 30]
    predicted_trajectories[:, t, 1] = current_positions[:, 1] + vel_y_avg * (t+1) * timestep  # shape [8, 30]

ades, fdes, error = ADE_FDE_calculation(predicted_trajectories)
cv_predicted_trajectories = PredictionModel(
    "Constant Velocity Model", 
    predicted_trajectories.copy(), 
    ades.copy(), 
    fdes.copy(),
    error.copy()
)

Agent 22 - ADE: 0.1429 FDE: 0.2534
Agent 15 - ADE: 1.7832 FDE: 6.0368
Agent 4 - ADE: 3.9185 FDE: 9.9969
Agent 5 - ADE: 2.9272 FDE: 7.1043
Agent 9 - ADE: 1.4494 FDE: 4.7071
Agent 14 - ADE: 0.855 FDE: 1.9633
Agent 6 - ADE: 0.4247 FDE: 1.0645
Agent 7 - ADE: 0.3868 FDE: 0.6687

Mean ADE: 1.486 Mean FDE: 3.9744


Q2.Using constant acceleration model to predict the future (1s,2s,3s) trajectories of the target agent and calculate the ADE and FDE. 

In [8]:
"""
    #Constant Acceleration Model#

    ca_predicted_trajectories instance is created 
    by using the average acceleration of the first 
    10 timesteps to predict the future positions.
"""

# Aquire the velocity information on current time step
vel_x = agents_to_predict[:, 10, 7] # shape [8, 30]
vel_y = agents_to_predict[:, 10, 8] # shape [8, 30]

# Use the average acceleration to predict the future positions
for t in range(predict_horizon):
    delta_t = (t + 1) * timestep
    predicted_trajectories[:, t, 0] = current_positions[:, 0] + vel_x * delta_t + 0.5 * acc_x_avg * delta_t**2
    predicted_trajectories[:, t, 1] = current_positions[:, 1] + vel_y * delta_t + 0.5 * acc_y_avg * delta_t**2

ades, fdes, error = ADE_FDE_calculation(predicted_trajectories)
ca_predicted_trajectories = PredictionModel(
    "Constant Acceleration Model", 
    predicted_trajectories.copy(), 
    ades.copy(), 
    fdes.copy(),
    error.copy()
)

Agent 22 - ADE: 0.9279 FDE: 2.5128
Agent 15 - ADE: 2.4521 FDE: 7.2758
Agent 4 - ADE: 1.8632 FDE: 5.4179
Agent 5 - ADE: 0.1984 FDE: 0.3739
Agent 9 - ADE: 0.9179 FDE: 1.1098
Agent 14 - ADE: 0.6053 FDE: 1.3769
Agent 6 - ADE: 1.9796 FDE: 4.9389
Agent 7 - ADE: 0.2129 FDE: 0.8187

Mean ADE: 1.1447 Mean FDE: 2.9781


Using Constant Turn Rate and Velocity(CTRV) model to predict the future (1s,2s,3s) trajectories of the target agent and calculate the ADE and FDE. 

In [9]:
"""
    #Constant Turn Rate and Velocity(CTRV) Model#

    ctrv_predicted_trajectories instance is created 
    by using the average velocity and the yaw rate of 
    the first 10 timesteps to predict the future positions.
"""

current_heading = agents_to_predict[:, 10, 6].copy()
predicted_trajectories_yaw = np.zeros((agents_to_predict.shape[0], predict_horizon, 2))  # shape [8, 30, 2]

# Calculate the average yaw rate of the agents
# agents_to_predict[:, :, 6] = np.radians(agents_to_predict[:, :, 6])  # Convert the heading to radians
yaw_rate = np.diff(agents_to_predict[:, his_start:11, 6], axis=1) / timestep # shape [8, 10]
average_yaw_rate = np.mean(yaw_rate, axis=1) # shape [8]

for t in range(predict_horizon):
    delta_x = vel_x_avg * np.cos(current_heading) * timestep
    delta_y = vel_y_avg * np.sin(current_heading) * timestep
    predicted_trajectories_yaw[:, t, 0] = predicted_trajectories_yaw[:, t-1, 0] + delta_x if t > 0 else current_positions[:, 0] + delta_x
    predicted_trajectories_yaw[:, t, 1] = predicted_trajectories_yaw[:, t-1, 1] + delta_y if t > 0 else current_positions[:, 1] + delta_y
    # Update the heading using the average yaw rate
    current_heading += average_yaw_rate * timestep

predicted_trajectories = predicted_trajectories_yaw
ades, fdes, error = ADE_FDE_calculation(predicted_trajectories)
ctrv_predicted_trajectories = PredictionModel(
    "Constant Turn Rate and Velocity Model", 
    predicted_trajectories.copy(), 
    ades.copy(), 
    fdes.copy(),
    error.copy()
)


Agent 22 - ADE: 3.3064 FDE: 6.4169
Agent 15 - ADE: 3.6645 FDE: 8.0741
Agent 4 - ADE: 4.8353 FDE: 11.7607
Agent 5 - ADE: 2.9274 FDE: 7.105
Agent 9 - ADE: 1.3404 FDE: 4.5349
Agent 14 - ADE: 2.1258 FDE: 4.241
Agent 6 - ADE: 0.7992 FDE: 1.3506
Agent 7 - ADE: 1.3407 FDE: 2.4844

Mean ADE: 2.5425 Mean FDE: 5.7459


Briefly discuss the observations from your results. 

If you want to visualize the scenario here are some simple demo codes for you.

    'TYPE_FREEWAY': 1,
    'TYPE_SURFACE_STREET': 2,
    'TYPE_BIKE_LANE': 3,

    # for roadline
    'TYPE_UNKNOWN': -1,
    'TYPE_BROKEN_SINGLE_WHITE': 6,
    'TYPE_SOLID_SINGLE_WHITE': 7,
    'TYPE_SOLID_DOUBLE_WHITE': 8,
    'TYPE_BROKEN_SINGLE_YELLOW': 9,
    'TYPE_BROKEN_DOUBLE_YELLOW': 10,
    'TYPE_SOLID_SINGLE_YELLOW': 11,
    'TYPE_SOLID_DOUBLE_YELLOW': 12,
    'TYPE_PASSING_DOUBLE_YELLOW': 13,

    # for roadedge
    'TYPE_ROAD_EDGE_BOUNDARY': 15,
    'TYPE_ROAD_EDGE_MEDIAN': 16,

    # for stopsign
    'TYPE_STOP_SIGN': 17,

    # for crosswalk
    'TYPE_CROSSWALK': 18,

    # for speed bump
    'TYPE_SPEED_BUMP': 19,
    
    # for driveway
    'TYPE_DRIVEWAY': 20,

In [10]:
"""
    This cell visualizes the predicted trajectories, history,
    and ground truth of each agent on the map.
"""

def visualize_trajectories(predicted_trajectories, color):
    """
        Visualize the predicted trajectories for each agent
        with the specified color.
        
        Args:
        - predicted_trajectories: PredictionModel
        - color: str

        Returns:
        - None
    """

    trajectory = predicted_trajectories.prediction

    for idx in range(tracks.shape[0]): 
        agent_traj = trajectory[idx] # shape [30, 2]
        # Calculate the range of the current trajectory (min and max coordinate values)
        min_x, max_x = min(agent_traj[:, 0]), max(agent_traj[:, 0])
        min_y, max_y = min(agent_traj[:, 1]), max(agent_traj[:, 1])
        x_range = (min_x - margin, max_x + margin)
        y_range = (min_y - margin, max_y + margin)
        # Get the current axis limits
        current_xlim = axs[idx].get_xlim()
        current_ylim = axs[idx].get_ylim()
        # Let the bigger range be the new range
        new_x_min = min(x_range[0], current_xlim[0])
        new_x_max = max(x_range[1], current_xlim[1])
        new_y_min = min(y_range[0], current_ylim[0])
        new_y_max = max(y_range[1], current_ylim[1])
        # Set the new axis limits
        axs[idx].set_xlim(new_x_min, new_x_max)
        axs[idx].set_ylim(new_y_min, new_y_max)
        # Label the corresponding prediction model
        label = 'prediction'
        if predicted_trajectories.name == 'Constant Velocity Model':
            label = 'CV prediction'
        elif predicted_trajectories.name == 'Constant Acceleration Model':
            label = 'CA prediction'
        elif predicted_trajectories.name == 'Constant Turn Rate and Velocity Model':
            label = 'CTRV prediction'
        axs[idx].plot(agent_traj[:, 0], agent_traj[:, 1], color, label=label)
        axs[idx].legend(loc='upper right')

# Global variables initialization
margin = 15  # Margin for the axis limits
history_trajectories = agents_to_predict[:, :10, :2]  # shape [8, 10, 2]
fig, axs = plt.subplots(3, 3, figsize=(15, 15)) # Create a 3x3 grid of subplots
axs = axs.flatten()

for idx, track_id in enumerate(tracks[:8]):
    axs[idx].set_facecolor('xkcd:grey')
    axs[idx].set_title(f'Agent {track_id}')
    start_point = current_positions[idx] # shape [2]
    # Set the initial axis limits
    agent_hist = history_trajectories[idx] # shape [10, 2]
    agent_gt = gt_future_position[idx] # shape [30, 2]
    all_gt = np.concatenate((agent_hist, agent_gt), axis=0)  # shape [40, 2]
    min_x, max_x = min(all_gt[:, 0]), max(all_gt[:, 0])
    min_y, max_y = min(all_gt[:, 1]), max(all_gt[:, 1])
    axs[idx].set_xlim(min_x - margin, max_x + margin)
    axs[idx].set_ylim(min_y - margin, max_y + margin)

    # Subplot for the map, history, ground truth, and start point
    for polyline in road_polylines:
        map_type = polyline[0, 6]
        color = 'w' if map_type in [6, 7, 8] else 'xkcd:yellow' if map_type in [9, 10, 11, 12, 13] else 'k'
        linestyle = 'dashed' if map_type in [6, 9, 10] else 'solid' if map_type in [7, 8, 11, 12] else 'dotted' if map_type == 13 else 'solid'
        axs[idx].plot(polyline[:, 0], polyline[:, 1], color=color, linestyle=linestyle, linewidth=1)
    axs[idx].plot(agent_hist[:, 0], agent_hist[:, 1], 'g', label='history')  # Use green line to plot history
    axs[idx].plot(agent_gt[:, 0], agent_gt[:, 1], 'g--', label='groud truth')  # Use green dashed line to plot ground truth
    axs[idx].scatter(start_point[0], start_point[1], c='g')  # Use green dot to plot start point
    axs[idx].legend(loc='upper right')

# Visualize the predicted trajectories
visualize_trajectories(cv_predicted_trajectories, 'b')
visualize_trajectories(ca_predicted_trajectories, 'r')
visualize_trajectories(ctrv_predicted_trajectories, 'm')

# Save the visualization as 'viz.png'
plt.tight_layout()
filename = 'data/Task 2/viz.png'
plt.savefig(filename)
plt.close()


In [ ]:
"""
    This cell visualizes the displacement errors of each agent,
    and displays the mean ADE and FDE for each time range.
"""

# Global variables initialization
cv_errors = cv_predicted_trajectories.distances # shape (8, 30)
ca_errors = ca_predicted_trajectories.distances # shape (8, 30)
ctrv_errors = ctrv_predicted_trajectories.distances # shape (8, 30)
timesteps = np.arange(11, 11+predict_horizon)
fig, axs = plt.subplots(3, 1, figsize=(8, 10))
models = ['CV Model', 'CA Model', 'CTRV Model']
errors = [cv_errors, ca_errors, ctrv_errors]
# Get rid of the useless index
useless_idx = 0
for idx, track in enumerate(tracks):
    if idx == 22:
        useless_idx = idx
        break

# Plot the displacement errors for each model
for ax, model, error in zip(axs, models, errors):
    for i in range(error.shape[0]):
        if i == useless_idx:
            continue
        ax.plot(timesteps, error[i, :], label=f'Agent - {tracks[i]}')
    ax.set_title(f'{model} Displacement Errors')
    ax.set_xlabel('Timestep')
    ax.set_ylabel('Displacement Error')
    # Vertical dashed lines for each second
    for line in range(10, 41, 10):
        ax.axvline(x=line, color='k', linestyle='--', linewidth=1)
    # Delete the useless agent error
    for agent_idx in range(error.shape[0]):
        if agent_idx == useless_idx:
            error = np.delete(error, agent_idx, axis=0)
    # Display the mean ADE and FDE for each time range
    for line in range(10, 31, 10):
        mean_ade = np.round(np.mean(error[:,:line]), 4)
        ax.plot([], [], ' ', label=f'{int(line/10)} sencond(s) mean ADE: {mean_ade}')
    for line in range(10, 31, 10):
        mean_fde = np.round(np.mean(error[:,line-1]), 4)
        ax.plot([], [], ' ', label=f'{int(line/10)} sencond(s) mean FDE: {mean_fde}')
    ax.legend(loc='upper left', ncol=2)    

# Save the visualization as 'metrics.png'
plt.tight_layout()
filename = 'data/Task 2/metrics.png'
plt.savefig(filename)
plt.close()
